# VacationPy
----


In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# ignore some warnings to make things look nicer
import warnings; warnings.simplefilter('ignore')

### Store WeatherPy Results into DataFrame
Loading the csv exported in Part I to a DataFrame

In [ ]:
csv_path = "../output_data/cities.csv"

# import the cities.csv file as a DataFrame
cities = pd.read_csv(csv_path)
cities.head()


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# access maps with unique API key
gmaps.configure(api_key=g_key)

In [ ]:
# store latitude and longitude in locations
locations = cities[["lat", "lng"]].astype(float)

# convert humidity values to float
humidity = cities["humidity"].astype(float)

In [ ]:
# Plot Heatmap

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# print description of map and instructions
print("Below is a gmap heatmap based on humidity in a list of cities. Zoom in (+ on bottom right) for more detail.")
print("-----------------------------------------------------------------------------------------------------------")

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# create new dataframe with my ideal weather conditions (listed below)
# temperature criteria 50 < temp < 75 F
# humidity criteria </= 60%
# cloudiness </= 70%
# wind speed </= 8 mph

ideal_cities = cities[(cities['max temp']>= 50) & (cities['max temp']<70 )& (cities['humidity']<=60) & (cities['cloudiness']<=70) & (cities['wind speed']<= 8)]

print(f"After filtering for ideal weather conditions, there are {len(ideal_cities['city'])} cities in the dataframe (shown below).")
ideal_cities.head(30)
 

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# renaming/storing variables per instructions
hotel_df = ideal_cities

# set up additional columns to hold information
hotel_df['hotel name'] = ""


In [ ]:
# define place search variables for google search
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print(f"Beginning Data Retrieval")
print(f"-------------------------------")

# Loop through the list of cities and perform a request for data on each
# Make a request for each of the indices

for index, row in hotel_df.iterrows():

    # get latitude from df
    lat = row['lat']
    
    # get longitude from df
    lng = row['lng']
    
    # get city from df
    city = row['city']
    
    # get country from df
    country = row['country']

    # add lat & lng to params dict
    target_coordinates = f"{lat}, {lng}"
        
     # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}

    # assemble url and make API request
    print(f"Retrieving Results for {city}, {country} at latitude {lat} and longitude {lng}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:      
        hotel_df.loc[index, 'hotel name'] = results[0]['name']
              
    except:
        print(f"No hotel found within radius of {city},{country}... skipping.")
        hotel_df.loc[index, 'hotel name'] = "NA"
    pass



In [ ]:
# drop rows with no hotel info

# find rows with missing values 
indexNames = hotel_df[hotel_df["hotel name"] == "NA" ].index

# delete these row indexes from dataFrame
hotel_df.drop(indexNames, inplace=True)

final_list = len(hotel_df['hotel name'])
print(f"There are {final_list} hotels available in the regions with ideal weather conditions.")
print("These hotels are listed in the dataframe below.")

hotel_df.head(30)

In [ ]:
humidity_values = hotel_df["humidity"]
max_humidity = humidity_values.max()

print(f"The maximum humidity value in the list of cities with ideal weather conditions is {max_humidity}.")
print("This humidity value should be the max_intensity value in the new heat_layer provided in the next cell.")

In [ ]:
# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
location = hotel_df[["lat", "lng"]]
# define marker layer
marker_layer = gmaps.marker_layer(location, info_box_content=hotel_info)

# Create humidity heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=60,
                                 point_radius=3)

# Add marker layer ontop of heat map
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# print description of map and instructions
print("Below is a gmap heatmap based on humidity in a list of cities.")
print("Each pin contains the name, city, and country of the nearest hotel within the city coordinates.")
print("Zoom in (+ on bottom right) for more detail.")
print("-----------------------------------------------------------------------------------------------------------")


# display figure
fig

# additional reference used to get this done: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html